In [ ]:
%matplotlib inline
import sys,os
import matplotlib.pyplot as plt
import torch
import spartan2.spartan as st

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] ="0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
st.config(st.engine.SINGLEMACHINE)

In [ ]:
# load time data
data = st.loadTensor(name = "example_time", path = "inputData/", col_types = [float, float, float], hasvalue=True)

In [ ]:
time_series = data.toTimeseries(attrlabels=['ecg_low', 'ecg_high'])

In [ ]:
time_series.attrlists.shape
seg=st.series_segmentation.create(time_series,st.sseg_policy.RPeaks,"slide")

ts_segs=seg.run(360,120,136,None)
print(ts_segs.shape)

In [ ]:
plt.plot(ts_segs[2][1,:])

In [ ]:
# create a beatgan model

ad_model = st.anomaly_detection.create(time_series, st.ad_policy.BEATGAN, "my_beatgan_model")

In [ ]:
### param for train model
param = {
    'network':'CNN',
    'net_type':'gru',
    'layers':1,
    'seq_len': 256,
    'input_size':2,
    'hidden_size':100,
    'rep_size':20,
    'batch_size':64,
    'max_epoch':5,
    'lr': 0.01,
    'lambda':1
}

In [ ]:
ad_model.init_model(param,device)

In [ ]:
ad_model.train(ts_segs)

In [ ]:
ad_model.test(ts_segs)

In [ ]:
ad_model.export("")

In [ ]:
## param for run
param_run = {
    'model_path':"",
    'network':'CNN',
    'net_type':'gru',
    'layers':1,
    'seq_len': 256,
    'input_size':2,
    'hidden_size':100,
    'rep_size':20,
    'batch_size':64,
    'max_epoch':5,
    'lr': 0.01,
    'lambda':1
}
### run model (load form model_path)
beatgan,res=ad_model.run(None,time_series,param_run,device)

In [ ]:
### run model from cache
ad_model.run(beatgan,time_series,param_run,device)